## Implementing Bag of Visual Words

### Installing all needed packages

In [ ]:
%pip install opencv-contrib-python opencv-python
%pip install matplotlib
%pip install scipy
%pip install joblib

## Import images

In [ ]:
import numpy as np
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import os

#loads images into a list
#image_directory = 'images/'
#image_directory = 'Results-Frames-Uncropped/'

#image_list = [Image.open(os.path.join(image_directory, img)) for img in os.listdir(image_directory)]

image_list=[]

#make sure to change this depending on how many images there are
for i in range(0,869,1):
    #img_data = PIL.Image.open('images/'+str(i)+'.png' )
    img_data = PIL.Image.open('Results-Frames-Cropped//'+str(i)+'.png' )
    #img_data = PIL.Image.open('cropped-images/Crop-'+str(i)+'.png' )
    temp=np.array(img_data)
    image_list.append(temp)
image_data=np.array(image_list)

#print(image_list[1])
#print(image_list[2])

#stores images into numpy arrays
##image_array_list = [np.array(img) for img in image_list]
#Put them all into one array
#image_array = np.stack(image_array_list)


imgplot = plt.imshow(image_list[50])
#imgplot = plt.imshow(image_list[200])

## Convert images to grayscale

In [ ]:
import cv2

# convert images to grayscale
bw_images = []
for img in image_list:
    # if RGB, transform into grayscale
    if len(img.shape) == 3:
        bw_images.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    else:
        # if grayscale, do not transform
        bw_images.append(img)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(bw_images[50], cmap='gray')
plt.show()

## Extract Visual features

ORB

In [ ]:
# Feature Extraction using ORB
extractor = cv2.ORB_create(nfeatures=2000)

# initialize lists where we will store *all* keypoints and descriptors
keypoints = []
descriptors = []

for img in bw_images:
    # extract keypoints and descriptors for each image
    img_keypoints, img_descriptors = extractor.detectAndCompute(img, None)
    keypoints.append(img_keypoints)
    descriptors.append(img_descriptors)

FAST

In [ ]:
# Feature Extraction using FAST
# I get detect the feautures but still need to figure out how to
# do the rest of the analysis given that I don't have descriptors
extractor = cv2.FastFeatureDetector_create()
extractor.setNonmaxSuppression(False)

# initialize lists where we will store *all* keypoints and descriptors
keypoints = []
descriptors = []

for img in bw_images:
    # extract keypoints and descriptors for each image
    img_keypoints = extractor.detect(img, None)
    keypoints.append(img_keypoints)
    #descriptors.append(img_descriptors)

SURF

In [ ]:
# Feature Extraction using SIFT
#SURF is patented, so neeed to play around with the version of opencv or set 
#opencv-conrib support enabled to use it

#extractor =cv2.xfeatures2d.SURF_create(400)
extractor = cv2.xfeatures2d.SURF_create()    
# Only features, whose hessian is larger than hessianThreshold are retained by the detector

# initialize lists where we will store *all* keypoints and descriptors
keypoints = []
descriptors = []

for img in bw_images:
    # extract keypoints and descriptors for each image
    img_keypoints, img_descriptors = extractor.detectAndCompute(img, None)
    keypoints.append(img_keypoints)
    descriptors.append(img_descriptors)

SIFT

In [ ]:
# Feature Extraction using SIFT
extractor = cv2.xfeatures2d.SIFT_create()

# initialize lists where we will store *all* keypoints and descriptors
keypoints = []
descriptors = []

for img in bw_images:
    # extract keypoints and descriptors for each image
    img_keypoints, img_descriptors = extractor.detectAndCompute(img, None)
    keypoints.append(img_keypoints)
    descriptors.append(img_descriptors)

Now we start preparing the data for the clustering

In [ ]:
#This code just removes images that did not have any features identified
#all images will most likely have something 
print(f"len before: {len(descriptors)}")
# initialize list to store idx values of records to drop
to_drop = []
for i, img_descriptors in enumerate(descriptors):
    # if there are no descriptors, add record idx to drop list
    if img_descriptors is None:
        to_drop.append(i)

print(f"indexes: {to_drop}")
# delete from list in reverse order
for i in sorted(to_drop, reverse=True):
    del descriptors[i], keypoints[i]

print(f"len after: {len(descriptors)}")

Visualise the features Extracted

In [ ]:
output_image = []
tempNum =0
for x in range(0,860,100):
    print(x)
    output_image.append(cv2.drawKeypoints(bw_images[x], keypoints[x], 0, (255, 0, 0), flags=0))
                                # flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))
    plt.imshow(output_image[tempNum], cmap='gray')
    tempNum+=1
    plt.show()

Building vocabulary

In [ ]:
import numpy as np

# select the same numbers in each run
np.random.seed(0)
# select 1000 random image index values
sample_idx = np.random.randint(0, len(image_list)+1, 95).tolist()
len(sample_idx)


# extract the sample from descriptors
# (we don't need keypoints)
descriptors_sample = []

for m in sample_idx:
    descriptors_sample.append(np.array(descriptors[m]))

In [ ]:
all_descriptors = []
# extract image descriptor lists
for img_descriptors in descriptors_sample:
    # extract specific descriptors within the image
    for descriptor in img_descriptors:
        all_descriptors.append(descriptor)
# convert to single numpy array
all_descriptors = np.stack(all_descriptors)

In [ ]:
# check the shape
all_descriptors.shape

In [ ]:
# we can count the number of descriptors contained in descriptors to confirm
count = []
for img_descriptors in descriptors_sample:
    count.append(len(img_descriptors))
# here we can see the number of descriptors for the first five images
print(f"first five: {count[:5]}")
# and if we sum them all, we should see the 39893 from before
print(f"count all: {sum(count)}")

K-means

In [ ]:
# perform k-means clustering to build the codebook, takes some time tbh

from scipy.cluster.vq import kmeans

k = 500
iters = 1
#make sure to use .astype for ORB as it stores the description weird
codebook, variance = kmeans((all_descriptors).astype(float), k, iters)

Saving the vocab

In [ ]:
import joblib

# save number of clusters and codebook
# Joblib dumps Python object into one file
joblib.dump((k, codebook), "ORB-Cropped-codebook-500.pkl", compress=3)

Reading the vocab back into the workspace

In [ ]:
# load the visual features, number of clusters, and codebook
k, codebook = joblib.load("ORB-Cropped-codebook-500.pkl")

Now we can start to add all of our training images in to do the localisatin

In [ ]:
#### This is where you specify which images to test against
average=0
averageMin=0
tenFound=0
fiveFound=0
oneFound=0
count=1
for indexImage in range(1,869,10):
    print("testing for image: "+str(indexImage))
    train_image_list=[]
    #make sure to change this depending on how many images there are
    #for i in range(0,indexImage,indexImage+1):
        #img_data = PIL.Image.open('images/'+str(i)+'.png' )
    img_data = PIL.Image.open('cropped-images/Crop-'+str(indexImage)+'.png' )
    #img_data = PIL.Image.open('images/'+str(indexImage)+'.png' )
    temp=np.array(img_data)
    train_image_list.append(temp)
    image_data=np.array(train_image_list)

    #plt.imshow(train_image_list[0])
    #plt.show()
    # convert images to grayscale

    train_bw_images = []
    for img in train_image_list:
        # if RGB, transform into grayscale
        if len(img.shape) == 3:
            train_bw_images.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
        else:
            # if grayscale, do not transform
            train_bw_images.append(img)
    #plt.imshow(train_bw_images[0], cmap='gray')
    #plt.show()


###### Feature Extraction using SIFT
#    extractor = cv2.xfeatures2d.SIFT_create()

    # initialize lists where we will store *all* keypoints and descriptors
#    train_keypoints = []
#    train_descriptors = []

#    for img in train_bw_images:
#       # extract keypoints and descriptors for each image
#        img_keypoints, img_descriptors = extractor.detectAndCompute(img, None)
#        train_keypoints.append(img_keypoints)
#        train_descriptors.append(img_descriptors)


##### feature Extraction using ORB
    extractor = cv2.ORB_create(nfeatures=2000)
#
    # initialize lists where we will store *all* keypoints and descriptors
    train_keypoints = []
    train_descriptors = []

    for img in train_bw_images:
        # extract keypoints and descriptors for each image
        img_keypoints, img_descriptors = extractor.detectAndCompute(img, None)
        train_keypoints.append(img_keypoints)
        train_descriptors.append(img_descriptors)

##### vector quantization
    from scipy.cluster.vq import vq
    import math

    visual_words = []
    for img_descriptors in descriptors:
        # for each image, map each descriptor to the nearest codebook entry
        img_visual_words, distance = vq(img_descriptors, codebook)
        visual_words.append(img_visual_words)

    #print(len(visual_words))
    # Adding my Training image into the set of visual words
    for img_descriptors in train_descriptors:
        # for each image, map each descriptor to the nearest codebook entry
        img_visual_words, distance = vq(img_descriptors, codebook)
        visual_words.append(img_visual_words)

    #print(len(visual_words))

###### Frequency Count
    k=500
    frequency_vectors = []
    for img_visual_words in visual_words:
        # create a frequency vector for each image
        img_frequency_vector = np.zeros(k)
        for word in img_visual_words:
            img_frequency_vector[word] += 1
        frequency_vectors.append(img_frequency_vector)
    # stack together in numpy array
    frequency_vectors = np.stack(frequency_vectors)

    frequency_vectors.shape

##### Running tf-idf

    N = 870 # N is the number of images, i.e. the size of the dataset

    # df is the number of images that a visual word appears in
    # we calculate it by counting non-zero values as 1 and summing
    df = np.sum(frequency_vectors > 0, axis=0)
    #idf = np.array()
    #for i in df:
    # if(df[i]>0):
        #    idf[i] = np.log(N/ df[i])
    # else:
        #   idf[i]=0
    idf = np.log(N/ df)
    idf.shape, idf[:5]
    #if you see negativs here chekc that you said the right size of the data set 2 cells above
    tfidf = frequency_vectors * idf
    tfidf.shape, tfidf[0][:5]

    #important to replace all the nan values with 0
    tfidf[np.isnan(tfidf)] = 0

    # Now we can check to see the similarity of images
    # cosine similarity
    from numpy.linalg import norm


##### Chi Squared 
    search_i=869
    #a = frequency_vectors[search_i]
   # b = frequency_vectors#tfidf  # set search space to the full sample

    a = tfidf[search_i]
    b = tfidf  # set search space to the full sample
    chi_similarity=[]
    for i in range(0,870):
        tempChi=0
        for d in range(0,500):
            #preventing a divide by 0 error
            if(a[d]!=0):
                tempChi+=((a[d]-b[i][d])**2)/a[d]
        chi_similarity.append(tempChi)
    chi_similarity=np.array(chi_similarity)
   # print("Min Chi similarity:", round(np.min(chi_similarity),1))
   # print("Max Chi similarity:", np.max(chi_similarity))
   # print("Probability"+str(chi_similarity[indexImage]))
    average=average+chi_similarity[indexImage]
   # print("running average: "+str(average/count))

##### Cosine Similarity
  #  search_i=869
    #a = frequency_vectors[search_i]#tfidf[search_i]
    #b = frequency_vectors#tfidf  # set search space to the full sample

   # a = tfidf[search_i]
   # b = tfidf  # set search space to the full sample

   # cosine_similarity = np.dot(a, b.T)/(norm(a) * norm(b, axis=1))
#    print("Min cosine similarity:", round(np.min(cosine_similarity),1))
#    print("Max cosine similarity:", np.max(cosine_similarity))
#    cosine_similarity
#    print("Probability"+str(cosine_similarity[indexImage]))
   # average=average+cosine_similarity[indexImage]
#    print("running average: "+str(average/count))


#### Normalised Euclidean Distance
    #testing Normalised Euclidean Distance
    #remember the higher the distancee the less similar the images are:
    #a = frequency_vectors
    #b = frequency_vectors
   # search_i=869
   # a = tfidf[search_i]
   # b = tfidf  # set search space to the full sample
   # euclid_similarity = []
   # for i in range(0,870):
   #     tempEuc=0
   #     for k in range(0,500):
   #         tempEuc+=((a[k]-b[i][k])**2)/500
   #     euclid_similarity.append(tempEuc)
   # euclid_similarity=np.array(euclid_similarity)
    #print("Min euclidean similarity:", round(np.min(euclid_similarity),1))
    #print("Max euclidean similarity:", np.max(euclid_similarity))
#    print("Probability"+str(euclid_similarity[indexImage]))
   # average=average+euclid_similarity[indexImage]
#    print("running average: "+str(average/count))
    
### Output Results

    top_k = 5
    #here we are sorting from highest to lowest
    #use this if you are looking at the angle between histograms
    #idx = np.argsort(-cosine_similarity)[:top_k]
    idx = np.argsort(chi_similarity)[:top_k]
    #idx = np.argsort(euclid_similarity)[:top_k]
    print(idx)

    #here we are sorting from lowest to highest
    #use this if you are looking at distance between histograms
    #idx = np.argsort(euclid_similarity)[:top_k]
    #idx
    #print(cosine_similarity[idx[1]])
    #print(euclid_similarity[idx[1]])
    print(chi_similarity[idx[1]])

    for g in idx:
        min=869
        if(abs(idx[1]-indexImage)<min):
            min=abs(idx[1]-indexImage)
    if min<=10:
        tenFound=tenFound+1
    if min<=5:
        fiveFound=fiveFound+1
    if min<=2:
        oneFound=oneFound+1
    averageMin=averageMin+min
    #print("current Min Value: "+str(min))
    #print("average Min Value: "+str(averageMin/count))
    count =count+1
print("end average: "+str(average/(count-1)))
print("within One: "+str(oneFound))
print("within five: "+str(fiveFound))
print("within ten: "+str(tenFound))

The code above is a culmination of the code shown below:

In [ ]:
# vector quantization
from scipy.cluster.vq import vq

visual_words = []
for img_descriptors in descriptors:
    # for each image, map each descriptor to the nearest codebook entry
    img_visual_words, distance = vq(img_descriptors, codebook)
    visual_words.append(img_visual_words)


In [ ]:
# let's see what the visual words look like for image 0
visual_words[0][:5], len(visual_words[0])

In [ ]:
# the centroid that represents visual word 84 is of dimensionality...
codebook[84].shape  # (all have the same dimensionality)

Frequency count

In [ ]:
frequency_vectors = []
for img_visual_words in visual_words:
    # create a frequency vector for each image
    img_frequency_vector = np.zeros(k)
    for word in img_visual_words:
        if word<199:
            img_frequency_vector[word] += 1
    frequency_vectors.append(img_frequency_vector)
# stack together in numpy array
frequency_vectors = np.stack(frequency_vectors)

In [ ]:
frequency_vectors.shape

In [ ]:
# we know from above that ids 84, 22, 45, and 172 appear in image 0
for i in [84,  22,  45, 172]:
    print(f"{i}: {frequency_vectors[0][i]}")

In [ ]:
plt.bar(list(range(k)), frequency_vectors[0])
plt.show()

run tf-idf

In [ ]:
# N is the number of images, i.e. the size of the dataset
N = 869

# df is the number of images that a visual word appears in
# we calculate it by counting non-zero values as 1 and summing
df = np.sum(frequency_vectors > 0, axis=0)

In [ ]:
df.shape, df[:5]

In [ ]:
idf = np.log(N/ df)
idf.shape, idf[:5]

In [ ]:
tfidf = frequency_vectors * idf
tfidf.shape, tfidf[0][:5]

In [ ]:
plt.bar(list(range(k)), tfidf[0])
plt.show()

In [ ]:
search_i = 49

plt.imshow(bw_images[search_i], cmap='gray')
plt.show()

Again, for reference, our cosine similarity range is $[0,1]$, and it is calculated as follows:

$$cossim(A,B)= cos(\theta)=\frac{A \cdot B}{||A|| \space ||B||}$$

In [ ]:
# cosine similarity
from numpy.linalg import norm

a = tfidf[search_i]
b = tfidf  # set search space to the full sample

cosine_similarity = np.dot(a, b.T)/(norm(a) * norm(b, axis=1))
print("Min cosine similarity:", round(np.min(cosine_similarity),1))
print("Max cosine similarity:", np.max(cosine_similarity))

This is testing Chi squared algorithm, still playing around though
 $$x^2=\sum_{i=1}^n(\frac{(a-b)^2}{a})$$

In [ ]:
#testing Chi Square Distance
# again the lower the distance the worse the comparison
a = tfidf[869]
b = tfidf  # set search space to the full sample
chi_similarity=[]
for i in range(0,869,1):
    tempChi=0
    for k in range(0,200):
        #preventing a divide by 0 error
        if(a[k]!=0):
            tempChi+=((a[k]-b[i][k])**2)/a[k]
    chi_similarity.append(tempChi)
chi_similarity=np.array(chi_similarity)
print("Min Chi similarity:", round(np.min(chi_similarity),1))
print("Max Chi similarity:", np.max(chi_similarity))

This is using normalised euclidean distance: 
$$D=\sqrt{\sum_{i=1}^n\frac{(a-b)^2}{n}}$$

In [ ]:
#testing Normalised Euclidean Distance
#remember the higher the distancee the less similar the images are:
a = tfidf
b = tfidf  # set search space to the full sample
euclid_similarity = []
for i in range(0,95):
    tempEuc=0
    for k in range(0,200):
        tempEuc+=((a[0][k]-b[i][k])**2)/200
    euclid_similarity.append(tempEuc)
euclid_similarity=np.array(euclid_similarity)
print("Min euclidean similarity:", round(np.min(euclid_similarity),1))
print("Max euclidean similarity:", np.max(euclid_similarity))

In [ ]:
#print(cosine_similarity.shape)
#print(euclid_similarity.shape)
print(chi_similarity.shape)

In [ ]:
#cosine_similarity
#euclid_similarity
chi_similarity

In [ ]:
top_k = 5
#here we are sorting from highest to lowest
#use this if you are looking at the angle between histograms
#idx = np.argsort(-euclid_similarity)[:top_k]

#here we are sorting from lowest to highest
#use this if you are looking at distance between histograms
idx = np.argsort(euclid_similarity)[:top_k]
idx

In [ ]:
euclid_similarity[idx[0]]